# Install unsloth

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

# Import Necessary Libraries

In [ ]:
from unsloth import FastLanguageModel, PatchDPOTrainer, is_bfloat16_supported
from trl import DPOTrainer, DPOConfig
import torch
import ast
import re
import textwrap
import gc
import os
from datasets import load_dataset

# Load And Bulid The Model

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
model_name = "unsloth/Qwen2.5-Coder-1.5B-Instruct"

PatchDPOTrainer()

print(f"Loading {model_name}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

# Load  Dataset

In [ ]:
dataset_file = "/content/groq_dpo_dataset .jsonl"

print(f"Loading Dataset from: {dataset_file}...")

def formatting_func(example):
    return {
        "prompt": example["prompt"],
        "chosen": example["chosen"],
        "rejected": example["rejected"],
    }

dataset = load_dataset("json", data_files=dataset_file, split="train")

original_columns = dataset.column_names
dataset = dataset.map(formatting_func, remove_columns=original_columns)
print(" Dataset loaded successfully!")

# Train The Model

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

gc.collect()
torch.cuda.empty_cache()

training_args = DPOConfig(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,

    warmup_ratio = 0.1,
    num_train_epochs = 3,
    learning_rate = 5e-6,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 10,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 42,
    output_dir = "dpo_outputs",

    gradient_checkpointing = True,

    beta = 0.1,
    max_length = max_seq_length,
    max_prompt_length = 1024,
)

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = training_args,
)

print("Starting Training (Low Memory Mode)...")
dpo_trainer.train()

# Test The Model

In [ ]:
def extract_definitions(code_str):
    names = []
    try:
        tree = ast.parse(code_str)
        for node in tree.body:
            if isinstance(node, ast.ClassDef):
                names.append(node.name)
            elif isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
                if not node.name.startswith('_'):
                    names.append(node.name)
    except SyntaxError:
        pass
    return names

In [ ]:
def check_google_style_strict(docstring, node_type=None, function_name=""):
    errors = []
    if not docstring:
        return ["Docstring is empty or missing."]

    if "Args:" in docstring:
        pattern = r"\s+[\w\*]+\s*\(.*\):\s+"
        if not re.search(pattern, docstring):
            errors.append("Bad 'Args' format. Expected indentation + 'name (type): description'.")

    is_class = (node_type == ast.ClassDef)
    is_init = (function_name == "__init__")

    if not is_class and not is_init:
        if "Returns:" not in docstring and "Yields:" not in docstring:
            errors.append("Missing 'Returns:' (or 'Yields:') section.")

    return errors

In [ ]:
def extract_code_block(text):
    pattern = r"```(?:python)?\s*(.*?)\s*```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

In [ ]:
FastLanguageModel.for_inference(model)
def generate_verified_docstring(model, tokenizer, code_snippet, max_retries=3):
    real_names = extract_definitions(code_snippet)

    system_instruction = """You are a strict Python Code Editor.
    Your goal: Rewrite the provided code to insert Google-Style docstrings.

    ### CRITICAL RULES:
    1. Output the **FULL COMPLETE CODE** (imports, classes, functions). Do NOT summarize.
    2. Output ONLY valid Python code inside ```python``` blocks.
    3. Format Args: `param_name (type): description`.
    4. **Classes** do NOT need a 'Returns' section.
    5. `__init__` does NOT need a 'Returns' section.
    """

    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": f"Here is the code to document:\n\n{code_snippet}"}
    ]

    for attempt in range(max_retries):
        print(f"  Attempt {attempt + 1}...", end=" ")

        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs,
                max_new_tokens=2048,
                do_sample=True,
                temperature=0.2,
                repetition_penalty=1.1,
                pad_token_id=tokenizer.eos_token_id
            )

        generated_tokens = outputs[0][inputs.shape[-1]:]
        full_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)

        generated_code = extract_code_block(full_output)

        if not generated_code:
            generated_code = full_output.strip()

        errors = []

        try:
            module = ast.parse(generated_code)
        except SyntaxError as e:
            errors.append(f"Syntax Error: {e}")

        if not errors:
            found_names = []
            for node in module.body:
                if isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.AsyncFunctionDef)):
                    found_names.append(node.name)
                    if node.name.startswith('_') and node.name != "__init__": continue

                    docstring_content = ast.get_docstring(node)

                    style_errors = check_google_style_strict(
                        docstring_content,
                        node_type=type(node),
                        function_name=node.name
                    )

                    if style_errors:
                        errors.append(f"In '{node.name}': {', '.join(style_errors)}")

            for name in real_names:
                if name not in found_names:
                    errors.append(f"Missing original definition '{name}'. YOU MUST RETURN THE FULL CODE.")

        if not errors:
            print("Success!")
            return generated_code
        else:
            print(f"Failed: {errors[0]}...")

            error_msg = f"Your code has errors: {errors}. \nCRITICAL: You must rewrite the **ENTIRE ORIGINAL CODE** with fixes. Do not output snippet only."

            messages.append({"role": "assistant", "content": full_output})
            messages.append({"role": "user", "content": error_msg})

    print("Max retries reached.")
    return generated_code

In [ ]:
complex_test_code = """
def calculate_velocity(distance, time):
    if time == 0: raise ValueError("Time cannot be zero")
    return distance / time
"""
meta_code = """
from typing import Any, Dict

class SingletonMeta(type):
    _instances: Dict[Any, Any] = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

class Database(metaclass=SingletonMeta):
    def __init__(self, connection_string: str):
        self.connection_string = connection_string
"""
async_code = """
import asyncio

class AsyncDatabaseConnection:
    def __init__(self, db_url: str):
        self.db_url = db_url
        self.connection = None

    async def __aenter__(self):
        print(f"Connecting to {self.db_url}...")
        await asyncio.sleep(0.1)  # Simulate connection
        self.connection = "Connected"
        return self.connection

    async def __aexit__(self, exc_type, exc, tb):
        print("Closing connection...")
        await asyncio.sleep(0.1)  # Simulate closing
        self.connection = None
"""
dynamic_code = """
from typing import Any

class LazyProxy:
    def __init__(self, target: Any):
        self._target = target

    def __getattr__(self, name: str) -> Any:
        print(f"Intercepting access to {name}")
        attr = getattr(self._target, name)

        if callable(attr):
            def wrapper(*args, **kwargs):
                print(f"Calling method: {name}")
                return attr(*args, **kwargs)
            return wrapper
        return attr
"""
chatgpt = """
def normalize_text(text, lower=True, remove_punctuation=False):
    if lower:
        text = text.lower()

    if remove_punctuation:
        import string
        text = text.translate(str.maketrans("", "", string.punctuation))

    return text.strip()
    """
print("Starting Strict Verification...\n")
print(generate_verified_docstring(model, tokenizer, complex_test_code))
print(generate_verified_docstring(model, tokenizer, meta_code))
print(generate_verified_docstring(model, tokenizer, async_code))
print(generate_verified_docstring(model, tokenizer, chatgpt))